In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# LAB: Generando nuestra propia API

## Introducción e instrucciones

El objetivo de esta práctica es simular nuestro propio servidor para poner a disposición nuestra API para un cliente interno. 

Vamos a utilizar un set de datos adaptados de una competencia de kaggle ya cerrada: https://www.kaggle.com/kevinmh/fifa-18-more-complete-player-dataset

En el dataset vamos a encontrar todos los jugadores de fútbol argentino que están en el juego fifa-18, y contaremos con las siguientes variables: 
- **ID**: un número único que identifica al jugador en toda la base.
- **full_name**: nombre completo del jugador.
- **age**
- **club**: del jugador
- **height_cm**
- **weight_kg**
- **puntaje_global**: puntaje que identifica la habilidad general del jugador.
- **potencia**: potencia física del jugador.
- **ritmo**: velocidad de aceleración del jugador.
- **disparos**: nivel de precisión y potencia de sus remates.
- **pases**: nivel de precisión en sus pases.
- **amagues**: nivel de habilidad para amagar a un rival.
- **defensa**: capacidad defensiva general del jugador.
- **físico**: estado físico del jugador (nos indicaría qué tan rápido se cansa)

Su **misión** es simular en esta notebook un **SERVIDOR** donde está alojada su API y, desde otra notebook (Clase_PIckle_shelve_flask_CLIENT) hacer los pedidos simulando que son un **CLIENTE**.

Su API va a hacer varias cosas: va a entrenar un modelo, va a realizar un gráfico de los coeficientes del modelo y, además, si le pasamos datos de un jugador nos va a devolver la predicción de su **puntaje_global**.

La API tiene que tener **tres funciones/endpoints**:

### **PRIMERA FUNCIÓN/ENDOPOINT:** entrenar el modelo
> La primera de esas funciones/endpoint entrena el modelo y tiene que recibir:
- (i) el dataframe con la base de jugadores argentinos "fifa_18_jugadores_argentinos_dos.csv" que se encuentra dentro de la carpeta Data (tienen que pasar la base como un json), 
- (ii) la lista de predictores, 
- (iii) la variable target, 
- (iii) una dirección donde guardar el modelo entrenado, 
- (iv) un nombre para darle al modelo y a los estimadores que necesiten usar, y 
- (v) una lista de alphas (van a usar  ElasticNet -eNetCv) y determinar el mejor alpha.

> El **output** de esta primera función/endpoint tiene que ser un json que contenga: 
- (i) el mejor alpha elegido por el modelo
- (ii) el resultado de train del modelo
- (iii) el resultado de test del modelo
- Además, tienen que usar la librería PICKLE para guardar el modelo entrenado y los estimadores que hayan utilizado.
          

### **SEGUNDA FUNCIÓN/ENDOPOINT:** coeficientes del modelo
> La segunda función/endpoint, va a grabar en la misma carpeta que el modelo una figura de los coeficientes del modelo entrenado. Los inputs tienen que incluir, por lo menos:
- (i) dirección donde está guardado el modelo
- (ii) nombre del modelo guardado

> El **output** de esta segunda función/endpoint tiene que ser: 
- (i) un json con los valores de los coeficientes
- (ii) un figura .jpg que se guarde en la carpeta donde está guardado el modelo. 

### **TERCERA FUNCIÓN/ENDOPOINT:** predicciones
> La tercera función/endpoint, va a realizar predicciones cuando nosotros le pasemos los datos de un nuevo jugador. Los inputs tiene que incluir, por lo menos: 
- (i) dirección donde está guardado el modelo
- (ii) nombre del modelo y estimadores
- (iii) features del jugador del cual queremos hacer la predicción sobre su puntaje total

> El **output** de esta tercera función/endpoint tiene que ser: 
- (i) un json con la predicción del puntaje total del jugador.

<br>  
#### ¡Manos a la obra!
<br>  

<img src="img/05_flask.jpg" alt="Drawing" style="width: 300px;"/>

<div id="caja9" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/kit_de_salida.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;"><label><b>Para tener en cuenta:</b> Tal como vimos en las notebooks de la práctica guiada y en los checkpoints, por un lado tienen que definir las funionces/endopoints de su API y, al final de la notebook, ejecutar la línea de código <b>app.run(host='0.0.0.0')</b>. Recuerden que esta línea hace que la notebook simule ser un servidor que está poniendo a disposición a nuestra API para que podamos acceder desde la otra notebook que es el cliente (usando la librería request). Y por último: mientras esté en ejecución esta línea de código, acuérdense de que no pueden ejecutar ninguna otra celda de esta notebook. </label></div>
</div>

***

Recuerden importar todas las librerías que su servidor va a necesitar para poner la API a disposición y para hacer entrenar el modelo y guardarlo a disco

In [ ]:
import pandas as pd
import json
import pickle
from flask import  Flask, request, jsonify, render_template

# Recuerden que en este entorno de "servidor" tenemos que tener importadas las librerías que necesitamos usar
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV as eNetCv
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

# Librerías y configuración para visualizaciones
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("talk")
sns.set_style('darkgrid')
sns.set_context(rc={'xtick.labelsize': 13,'ytick.labelsize': 13, 'axes.labelsize': 16, 'axes.titlesize': 18})

***

## **PRIMERA FUNCIÓN/ENDOPOINT:** entrenar el modelo

In [ ]:
# Iniciamos nuestra API
app = Flask('Predictor de jugadores')

In [ ]:
# Iniciamos nuestro primera función asociada a su endpoint. Usamos el método POST ya que vamos a enviar información
# al servidor: nuestro modelo entrenado junto con sus estimadores.
@app.route('/entrenar_modelo',methods=['POST'])
def entrenar_modelo():
    
    # la función "request.get_json" de Flask para capturar la información que le envíemos a la API
    data = request.get_json(force=True)
    
    # Separamos la información de data y usamos json.loads() para transformar el dataframe que está en formato
    # json a un diccionario y luego lo convertimos en un DataFrame.
    df=pd.DataFrame(json.loads(data['base']))
    columns_name=data['lista_predictores']
    target_name=data['target']
    alphas_modelo=data['alphas']
    directorio_modelo=data['directory']
    nombre_modelo=data['name']
        
    # Separamos las "X" y la "y" para entrenar nuestro modelo
    X = df[columns_name]
    y = df[target_name]
    
    # Hacemos el train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=2020)
    
    # Hacemos el cross validation para ver el mejor alpha y entrenamos el modelo
    k=KFold(n_splits=3, shuffle=True, random_state=45)
    modelo = eNetCv(alphas=alphas_modelo, cv = k)
    
    # Escalamos los datos para que estén todos en la misma unidad (dado que estamos incluyendo la edad,
    # peso y altura, por ejemplo)
    scalador=StandardScaler()
    X_train_sc=scalador.fit_transform(X_train)
    X_test_sc=scalador.transform(X_test)
    
    # Entrenamos el modelo usando los datos escalados
    modelo.fit(X_train_sc, y_train)
        
    # Evaluamos el R2 de train y de test
    result_train=r2_score(y_train,modelo.predict(X_train_sc))
    result_test=r2_score(y_test,modelo.predict(X_test_sc))
        
    # Guardamos el modelo entrenado en una carpeta que se llama "modelos" que esté a la misma altura que la notebook
    dir_=directorio_modelo+'/'+nombre_modelo+'.pkl'
    with open(dir_, 'wb') as modelo_pkl:
        pickle.dump(modelo, modelo_pkl)
        
    # Guardamos también el escalador entrenado, ya que lo vamos a tener que aplicar a los casos nuevos que le pasemos
    dir_scaler=directorio_modelo+'/'+nombre_modelo+'_scaler.pkl'
    with open(dir_scaler, 'wb') as scaler_pkl:
        pickle.dump(scalador, scaler_pkl)
        
    
    # La función devuelva el alpha elegido por el modelo, el resultado de train y el de test.
    return {'alpha_elegido':float(modelo.alpha_),'resultado_train':float(result_train), 'resultado_test':float(result_test)}

***

## **SEGUNDA FUNCIÓN/ENDOPOINT:** coeficientes del modelo

In [ ]:
# Iniciamos nuestro segunda función asociada a su endpoint. Usamos el método POST ya que vamos a enviar información
# al servidor: la figura con los coeficientes de nuestro modelo entrenado
@app.route('/plot_coeficientes',methods=['POST'])
def plotear_coeficientes():
    
    # la función "request.get_json" de Flask para capturar la información que le envíemos a la API
    data = request.get_json(force=True)
    
    # Separamos la información de data.
    direccion=data['direccion']
    name_modelo=data['nombre_modelo']
    colores=data['paleta_colores']
    size=data['tamano']
    predictores=data['predictores']
    
    # Levantamos el modelo que tenemos grabado en disco con el llamado anterior
    # (hay que tener en cuenta que tenemos que hacer el llamado anterior primero cuando estemos en la notebook
    # que simula ser el cliente).
    dir_input=direccion+'/'+name_modelo+'.pkl'
    with open(dir_input, 'rb') as modelo_pkl:
        modelo_load = pickle.load(modelo_pkl)
        
    # Levantamos los coeficientes ya calculados del modelo
    # Los ponemos en un dataframe junto con el nombre de los predictores.
    # Pasamos los coeficientes a valores absolutos y los ordenamos de mayor a 
    # menor para hacer el gráfico.
    coef=abs(pd.DataFrame({'coeficientes':modelo_load.coef_},index=predictores)).\
                           sort_values(by='coeficientes',ascending=False)
    
    
    # Generamos la figura y la guardamos
    fig, ((ax1)) = plt.subplots(1,1,gridspec_kw={'hspace': 0.45, 'wspace': 0.15},figsize=size)
    fig.suptitle("Coeficientes",y=0.96,x=0.135,fontsize=24,fontweight='bold')

    ax1 = sns.barplot(x="index", y="coeficientes", data=coef.reset_index(),ax=ax1,palette=colores)
    ax1.xaxis.set_label_text('Coeficientes')
    ax1.yaxis.set_label_text('Valores coeficientes')
    plt.close(); #usamos esta línea para que la figura no se imprima en pantalla
    
    # guardamos la figura donde se guardan los modelos
    dir_figura=direccion+'/'+name_modelo+'.jpg'
    fig.savefig(dir_figura,dpi=150)
    
    # devolvemos un json con los coeficientes. Dado que las API devuelven la información en formato
    # json, usamos el método de los dataframes ".to_json()" para poder retornar los datos de los
    # coeficientes.
    return coef.to_json()    

***

## **TERCERA FUNCIÓN/ENDOPOINT:** predicciones

In [ ]:
# Dado que en este endopint no vamos a guardar nada en el servidor, sino recibir información, usamos 
# el método GET. 
@app.route("/prediccion",methods=['GET'])
def predecir_puntaje():
    
    # Usamos request.args para tomar las query que le pasamos a la URL
    direccion=request.args['direccion']
    name_modelo=request.args['nombre_modelo']
    name_scaler=request.args['nombre_scalador']
    caso_to_predict=request.args['features']
    
    # Levantamos el modelo y el escalador que ya tenemos entrenado
    with open(direccion+'/'+name_modelo+'.pkl', 'rb') as modelo_pkl:
        modelo_load = pickle.load(modelo_pkl)
        
    with open(direccion+'/'+name_scaler+'.pkl', 'rb') as scaler_pkl:
        scaler_load = pickle.load(scaler_pkl)
        
    # re-escalamos los datos con el escalador entrenado (tengan en cuenta que el escalador va a estar
    # esperando una estructura como un DataFrame)
    scaled_case=scaler_load.transform(pd.DataFrame(json.loads(caso_to_predict),index=[0]))
    
    # realizamos la prediccion
    prediccion=modelo_load.predict(scaled_case)
    
    return {'prediccion':float(prediccion)}

***

## **Disponibilizamos nuestra API**
<br>  

<img src="img/forrest.jpg" alt="Drawing" style="width: 400px;"/>


In [ ]:
# Ahora ejecutamos esta línea de código que pone a disposición los tres endpoints que armamos arriba. 
# Es hora de ir a la otra notebook en la que simulamos ser un cliente y hacer los llamadados para cada
# uno de estos tres endopoints...
app.run(host='0.0.0.0', port= 5009)